In [1]:
import pandas as pd
import camelot
import time
import numpy as np
import re
import glob 
import re

In [545]:
# Read in original pdf file using camelot and transform it into dataframes.

# tables = camelot.read_pdf("raw.pdf", flavor='lattice', pages='1-end')

In [546]:
# Save parsed dataframes into csv.

# for i, table in enumerate(tables):
#     table.df.to_csv(f"data/{i}.csv", index=False, header=False)

# Read in saved CSV files

In [2]:
# Read in all csv files and save as list of dataframes.

filenames = glob.glob("data/*.csv")
onlynumbers = [filename[5:-4] for filename in filenames]
numbers = sorted([int(num) for num in onlynumbers])

dataframes = []
for num in numbers:
    dataframes.append(pd.read_csv(f"data/{num}.csv"))

# Remove the first three tables, which are not real data, but an example tables from the document.
dataframes = dataframes[3:]

default_columns = dataframes[0].columns.str.replace(r'\n','', regex=True)
default_columns = [col for col in default_columns if col != 'NO']

# Strip all \n from cell values
# Handle Error Cases
# Standardize Column format
for idx, dataframe in enumerate(dataframes):
    dataframe.replace(r'\n','', regex=True, inplace=True)
    dataframe.columns = dataframe.columns.str.replace(r'\n','', regex=True)
    
    dataframe.rename(columns={
        "*허가일*변경일" : "허가및변경일"
    }, inplace=True)
    
    dataframe.rename(columns={
        "NO 대표자" : "대표자"
    }, inplace=True)
    
    try:
        dataframes[idx] = dataframe[default_columns]
    except Exception as e:
        error_msg = e
        
        if str(error_msg) == "\"['용량(MW)', '허가및변경일'] not in index\"":
            dataframe.rename(columns={
                "Unnamed: 6" : "허가및변경일",
                "용량허가및변경(MW)일" : "용량(MW)"
            }, inplace=True)
            dataframes[idx] = dataframe[default_columns]
        else:
            print(f"Problem with Columns in Dataframe {idx} : {error_msg}")

In [3]:
# Concatenate all tables into a single master df

df = pd.concat(dataframes, ignore_index=True)

# Create column names

df.columns = ['owner', 'company', 'address', 'type', 'capacity_MW', 'approval_date', 'prep_time', 'note']

# Remove all rows with no cell values

df = df[~df.isnull().all(axis=1)]

In [4]:
quote_missing = df.approval_date.isna() & df.prep_time.notnull()
df.loc[quote_missing, "approval_date"] = df.loc[quote_missing, "approval_date"].fillna("“")

# Replace '“', '\"' values with proper cell values.

for column in df.columns:
    missing_index = df[(df[column] == '“') | (df[column] == '\"') | (df[column] == '‘“')].index.values.astype(int)
    for index in missing_index:
        df[column][index] = df[column][index-1]

df = df[~df.isnull().all(axis=1)]
df.reset_index(drop=True, inplace=True)

In [550]:
# Some "owner" cells include company names. 
# If owner cell has two words and the company cell is empty,
# transfer the second word from the owner cell to the company cell.

company_in_owner_col_filt = (df['owner'].str.split().str.len() > 1) & df['company'].isna()

companies_and_owners = df.loc[company_in_owner_col_filt, 'owner'].str.split()

owners = companies_and_owners.apply(lambda l: l[0])
companies = companies_and_owners.apply(lambda l: l[1])

df.loc[company_in_owner_col_filt, 'owner'] = owners
df.loc[company_in_owner_col_filt, 'company'] = companies


# Some "address" cells include company names. 

# If the company cell is empty and address cell is not, transfer the company name from the address cell to the company cell.

company_in_address_col_filt = df['approval_date'].notnull() & df['address'].notnull() & df['owner'].notnull() & df['company'].isna()

companies_and_adddress = df.loc[company_in_address_col_filt, 'address'].str.split()

companies = companies_and_adddress.apply(lambda l: l[0])
addresses = companies_and_adddress.apply(lambda l: " ".join(l[1:]))

df.loc[company_in_address_col_filt, 'company'] = companies
df.loc[company_in_address_col_filt, 'address'] = addresses



# Combine overflowing rows into a single row.

In [551]:
def concat(v1, v2):
    result = ""
    if v1 is not np.nan:
        result += str(v1)
    if result:
        result += " "
    if v2 is not np.nan:
        result += str(v2)
    if not result:
        result = np.nan
    return result

In [552]:
def roll_up(df):
    for idx in reversed(df[df["approval_date"].isna()].index):
        df.loc[idx-1] = df.loc[idx-1].combine(df.loc[idx], concat)
        df.drop(index=idx, inplace=True)
    df.reset_index(drop=True, inplace=True)

In [553]:
roll_up(df)

In [557]:
df[df.address.str.contains("포지리  1249", na=False)]

,owner,company,address,type,capacity_MW,approval_date,prep_time,note
1843,김철웅,이원신재생에너지복지마을,충남 태안군 이원면 포지...,태양광,30.0,2020-06-30,2023-03-31,NaN
2077,김병숙,한국서부발전,충남 태안군 이원명 포지...,태양광,45.0,2021-01-27,2023-01-31,사업준비기간연장
2653,신경철,이원신재생에너지복지마을,충남 태안군 이원면 포지...,태양광,30.0,2022-06-28,2023-03-31 대표자 변경,NaN


# Replace NaN in note column to "신규" (newly approved)

### Check if note section is in "prep_time" column and move them into "note" column.

In [559]:
pd.options.display.max_colwidth = None

df.prep_time.fillna("", inplace=True)

df_note_in_prep = df[~df.prep_time.str.contains(r"^(\d|-|\.|\s|‘|호기|년이?내?|개?월|'|"")+$", na=False)][3:]
df_note_in_prep.drop([217], axis=0, inplace=True)

df_note_in_prep



/var/folders/_c/dcjy1m9109bd_sq8l8j8zsdr0000gn/T/ipykernel_61593/1590430054.py:5: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df_note_in_prep = df[~df.prep_time.str.contains(r"^(\d|-|\.|\s|‘|호기|년이?내?|개?월|'|"")+$", na=False)][3:]


,owner,company,address,type,capacity_MW,approval_date,prep_time,note
129,양짜오,MPC율촌복합,전남 순천시 해룡면,LNG복합,884.5,‘11.12.26,.13.6월(GT)‘14.6월(ST),사업개시(‘14.4.29)
136,이길구,울산 4 복합화력발전(한국동서발전),울산 남구 남화동,LNG복합,"872(GT 2기,ST1기)",‘12.1.31,‘13.7월(GT)'14.7월(ST),NaN
137,김문덕,평택복합 2단계 발전,경기도 평택시 포승면,LNG복합,"868.5(GT2기, ST1기)",‘12.1.31,‘13.7월(GT)'14.10월(ST),NaN
141,정동화,(주)포스코건설,강원도 춘천시 석사동산73,집단에너지,463(GT1기ST1기),_,_,용량변경허가202→463
199,김홍권,수완에너지(주),광주광역시 광산구수완로 130,열병합(LNG),109,‘12.12.5,2013.3.1발전사업전환,NaN
219,김문덕,한국서부발전(주),"전남 영암군 삼호읍삼포리 1895,1898",태양광,13.296,'13.4.2,양수인가‘13.4.2,양도인:영암에프원태양광발전(주)
221,임승환,부산신호태양광발전(주),부산시 강서구르노삼성대로 61(신호동),태양광,20,'13.4.2,변경허가,"11MW,9MW를20MW대표자변경"
320,조석,한국수력원자력(주),경북 경주시 양북면봉길리 일원,원자력(S/T),"2,000",‘14.2.2,2015년1월20일+,사업준비기간변경
450,장만천,전주페이퍼,전북 전주시 덕진구 팔복로 59,바이오메스,14.95,2014-12-08,2014-12-31운영중,대표자변경
470,미쯔오카히데노리,고원풍력발전,강원 태백시 창죽동 산 63번지 일원,풍력,20.0,2015-01-14,2015-05-31 대표자변경,NaN


In [ ]:
def retrieve_note(prep):
    note = re.findall(r"^\d{4}\.?-?\d{1,2}\.?-?\d{1,2}(.*)", prep)[0].strip()
    return note

df_note_in_prep.note = df_note_in_prep.prep_time.apply(lambda x: retrieve_note(x))

df_note_in_prep
df.loc[df_note_in_prep.index, "note"] = df_note_in_prep.note

In [ ]:
df.loc[2653]

In [ ]:
df[df.address.str.contains("포지리  1249", na=False)]

In [ ]:
df.note.fillna('신규', inplace=True)
df.note = df.note.str.replace(" ", "") \
    .str.replace("nan", "신규") \
    .str.replace("nannan", "신규") \
    .str.replace("신규신규", "신규")


# Check non-Nan values in note to see if they are also onewly approved.

not_new = ['연장', '변경', '취득', '용량', '용량변경', '정정', '재발급', '인가', '양수', '추가', '합병', '증설', '양도', '개시', '반납', '취소', '축소', '준비기간', '조정', '법인', 'SRF', "→", '지번', '재교부', '전환', '수정', '발전', '종류', '공급', '폐기물', '배방읍', '구분']
# new = ['신규', '이내', '허가', '일부', '동의']

df.loc[~df.note.str.contains('|'.join(not_new)), "note"] = "신규"


In [ ]:
pd.options.display.max_colwidth=20

df[~df.note.str.contains('|'.join(not_new))].note.unique()

# Convert "approval_date" from df_new to datetime

In [ ]:
# Only select plants that are newly approved

df_new = df[df.note=='신규']

In [ ]:
df_new[df_new.address.str.contains("포지리  1249", na=False)]

In [ ]:
# Clean up approval_date

df_new.approval_date = df_new.approval_date.str.replace("`", "20", regex=False).str.replace("‘", "20", regex=False).str.replace("'", "20", regex=False).str.replace(".", "-", regex=False)
df_new.approval_date = df_new.approval_date \
                        .str.replace(r"^12-5$", "2011-12-05", regex=True) \
                        .str.replace(r"^3-5$", "2012-03-05", regex=True) \
                        .str.replace(r"^3-30$", "2012-03-30", regex=True) \
                        .str.replace(r"^4-26$", "2012-04-26", regex=True) \
                        .str.replace(r"^12-6-4$", "2012-06-04", regex=True)


In [ ]:
df_new['year'] = df_new.approval_date.str[:4]

month_regex = r"\d{4}-(\d{1,2})"

def string_to_month(string):
    month = re.findall(month_regex, string)[0]
    return month

df_new['month'] = df_new.approval_date.apply(lambda x: string_to_month(x))
df_new['month'] = df_new['month'].str.zfill(2)

df_new['cleaned_approval_time'] = df_new['year'] + "-" + df_new['month']
df_new['cleaned_approval_time'] = pd.to_datetime(df_new['cleaned_approval_time'], format="%Y-%m")

# Clean up "용량(MW)" (capacity) column into a consistent format

### 1. If the cell is in KW, change the number and unit to MW.

In [ ]:
df_in_kw = df_new[df_new['capacity_MW'].str.lower().str.contains("kw", na=False)]

# Identify which cells have multiple plants, so I can multiply the capacity by two later on.
has_multiple_plants = df_in_kw['capacity_MW'].str.contains("기")

# df.capacity_MW["kw" in df.capacity_MW.str.lower()]
has_10k = df_in_kw['capacity_MW'].str.contains("만")
has_1k = df_in_kw['capacity_MW'].str.contains("천")

df_in_kw.loc[has_10k, 'cleaned_capacity'] = df_in_kw['capacity_MW'].str.replace("만", "0000")
df_in_kw.loc[has_10k & has_1k, 'cleaned_capacity'] = df_in_kw['capacity_MW'].str.replace("만", "").str.replace("천", "000")

# Fill in empty 'cleaned_capacity' column with 'capacity_MW' values.
df_in_kw.cleaned_capacity = df_in_kw.cleaned_capacity.fillna(df_in_kw.capacity_MW)

# Convert capacity value into int and divide capacity by 1,000
df_in_kw.cleaned_capacity = df_in_kw.cleaned_capacity \
                                .str.lower() \
                                .str.replace("총", "") \
                                .str.replace("kw", "") \
                                .str.replace(",", "") \
                                .str.replace("7-8호기", "") \
                                .str.replace("2기", "")

df_in_kw.cleaned_capacity = df_in_kw.cleaned_capacity.astype(float)
df_in_kw.cleaned_capacity = df_in_kw.cleaned_capacity / 1000
df_in_kw[has_multiple_plants].cleaned_capacity = df_in_kw[has_multiple_plants].cleaned_capacity * 2


df_in_kw['capacity_MW'] = df_in_kw['cleaned_capacity']
df_in_kw.drop(columns=['cleaned_capacity'], inplace=True)

df_new.loc[df_in_kw.index, 'capacity_MW'] = df_in_kw['capacity_MW']


### 2. Check if there are other problems with the capacity_MW columns

In [ ]:
# Remove all commas and "MW" from values.
df_new.capacity_MW = df_new.capacity_MW.astype(str)\
                        .str.lower() \
                        .str.replace(",", "") \
                        .str.replace("mw", "") \
                        .str.replace("nan", "") \
                        .str.replace("!", "1") \
                        .str.replace(r"×?1기", "")

# Remove additional information from capacity_MW cells.
has_additional_info = df_new.capacity_MW.str.contains("\(") & df_new.capacity_MW.str.contains("\)")
df_new["capacity_MW"] = df_new["capacity_MW"].str.replace(r"[(].*[)]", "", regex=True)

# Additional manual cleaning for misread values by cross-referncig them with original pdf file.
df_new.capacity_MW = df_new.capacity_MW.str.replace("13900×22500", "36400") \
                                        .str.replace("834.3\(gt2기 st", "834.3") \
                                        .str.replace("13500×5기1000×3기380", "70880") \
                                        .str.replace("여수250군장122.9", "372.9") \
                                        .str.replace("126313.67.3", "1283.9") \
                                        .str.replace("555.17.2", "762.3") \
                                        .str.replace("19.99 3.0", "19.993") \
                                        .str.replace(" ", "") \
                                        .str.strip() \
                                        .str.replace("82219.2", "841.2") \

# There are three entries with empty capacity, where its 9.9 capacity is placed within the energy type. Fix these entires.
df_new.loc[df_new.capacity_MW == "", 'capacity_MW'] = df_new[df_new.capacity_MW == ""]["capacity_MW"].str.replace("", "9.9")


# Some capacity numbers are unreasonbly high, and may actually be in kw. Identify these entries.
df_new.capacity_MW = df_new.capacity_MW.astype(float)
df_new[df_new.capacity_MW>1000]
df_new.loc[df_new.capacity_MW >= 35000, 'capacity_MW'] = df_new[df_new.capacity_MW >= 35000]["capacity_MW"]/1000



# Clean "Type" from df_new

In [ ]:
df_new.type = df_new.type.str.replace(" ", "")

# Manual cleaning misread values by cross-referencing with original pdf.
df_new.company = df_new.company.str.replace("영암태 전", "영암태양광발전")
df_new.address = df_new.address.str.replace("양광발 전", "전")
df_new.loc[956, "type"] = "태양광"
df_new.loc[1667, "type"] = "태양광"
df_new.loc[2572, "type"] = "풍력"

### Simplify type into "cleaned_type" column.

In [ ]:
df_new['cleaned_type'] = np.nan

# df_new.loc[df_new.type.str.contains("열병합", na=False), 'cleaned_type'] = "coal"
# df_new.loc[df_new.type.str.contains("복합", na=False), 'cleaned_type'] = "coal"
df_new.loc[df_new.type.str.contains("가스", na=False), 'cleaned_type'] = "gas"
df_new.loc[df_new.type.str.contains("폐열", na=False), 'cleaned_type'] = "coal"
df_new.loc[df_new.type.str.contains("바이오", na=False), 'cleaned_type'] = "bioenergy"
df_new.loc[df_new.type.str.contains("목질계", na=False), 'cleaned_type'] = "bioenergy"
df_new.loc[df_new.type.str.contains("우드칩", na=False), 'cleaned_type'] = "bioenergy"
df_new.loc[df_new.type.str.contains("원자력", na=False), 'cleaned_type'] = "nuclear"
df_new.loc[df_new.type.str.contains("수력", na=False), 'cleaned_type'] = "hydro"
df_new.loc[df_new.type.str.contains("양수", na=False), 'cleaned_type'] = "hydro"
df_new.loc[df_new.type.str.contains("풍력", na=False), 'cleaned_type'] = "wind"
df_new.loc[df_new.type.str.contains("태양광", na=False), 'cleaned_type'] = "solar"
df_new.loc[df_new.type.str.contains("석탄", na=False), 'cleaned_type'] = "coal"
df_new.loc[df_new.type.str.contains("유연탄", na=False), 'cleaned_type'] = "coal"
df_new.loc[df_new.type.str.lower().str.contains("ng", na=False), 'cleaned_type'] = "gas"
# df_new.loc[df_new.type.str.lower().str.contains("srf", na=False), 'cleaned_type'] = "srf"
# df_new.loc[df_new.type.str.lower().str.contains("생활", na=False), 'cleaned_type'] = "srf"
# df_new.loc[df_new.type.str.lower().str.contains("폐기물", na=False), 'cleaned_type'] = "srf"
# df_new.loc[df_new.type.str.lower().str.contains("페기물", na=False), 'cleaned_type'] = "srf"
# df_new.loc[df_new.type.str.lower().str.contains("고체연료", na=False), 'cleaned_type'] = "srf"
# df_new.loc[df_new.type.str.lower().str.contains("고형연료", na=False), 'cleaned_type'] = "srf"
# df_new.loc[df_new.type.str.lower().str.contains("연료전지", na=False), 'cleaned_type'] = "hydrogen fuel cells"
# df_new.loc[df_new.type.str.lower().str.contains("조력", na=False), 'cleaned_type'] = "ocean"
# df_new.loc[df_new.type.str.lower().str.contains("해양", na=False), 'cleaned_type'] = "ocean"
# df_new.loc[df_new.type.str.lower().str.contains("스팀", na=False), 'cleaned_type'] = "steam"
# df_new.loc[df_new.type.str.lower().str.contains("증기", na=False), 'cleaned_type'] = "steam"
df_new.loc[df_new.cleaned_type.isna(), 'cleaned_type'] = "others"




In [ ]:
df_cleaned = df_new[['cleaned_type', 'capacity_MW', 'address', 'year', 'company', 'owner']]

# Clean up address

### Check repeating addresses to make sure they are not redundancies.

Following addresses are checked with original pdf to prove they are separate plants:
* 강원도  영월군  영월읍  정양리  540,  541-1,  541-2, 566,  567,  568,  569
* 전남  장흥군  장흥읍  삼산리  749-1번지  일원   
* 전남 나주시 영산동664-1 등지 (Multiple parks by same company in the same location (likely expansion)
* 강원도  영월군  상동읍  천평리  249번지  일원
* 인천시  남구  학익동  723번지
* 부산시  기장군  정관면  산단1로  83


Following addresses are pending investigations.
* 충남  아산시  둔포면  염작리  92-28 -> I have reached out to KOREC to clarify the situation. The data has not be treated and all addresses remain in df_new.
* 제주시  구좌읍  덕천리  산82 (Are these three separate solar farms? Or just duplication by mistake?)

Following addresses need to be treated to drop redundancy.
* 강원도  강릉시  강동면  모전리  253  및  254-1~2번지
* 전남 장흥군 장흥읍 삼산리 749-1번지 일원
* 경북  영천시  신녕면  화남리  산64,  군위군  고로면 화북리  산229-1번지  외  3필지 (Error from note being improperly filtered from earlier.)

In [ ]:
# Check repeating addresses to make sure they are not redundancies.

# Following addresses are checked with original pdf to prove they are separate plants:

df_cleaned.address.value_counts().head(55)

### Drop rows that are redundancies.

In [521]:
pd.options.display.max_colwidth = None

# Drop rows that are redundancies.

# 전남 장흥군 장흥읍 삼산리 749-1번지 일원
df_cleaned.drop([2654, 2655], axis=0, inplace=True)

# 강원도  강릉시  강동면  모전리  253  및  254-1~2번지    
df_cleaned.drop([1407, 2195, 2640], axis=0, inplace=True)

# 경북  영천시  신녕면  화남리  산64,  군위군  고로면 화북리  산229-1번지  외  3필지
df_cleaned.drop([1840, 1883], axis=0, inplace=True)

/var/folders/_c/dcjy1m9109bd_sq8l8j8zsdr0000gn/T/ipykernel_61593/4285434580.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned.drop([2654, 2655], axis=0, inplace=True)
/var/folders/_c/dcjy1m9109bd_sq8l8j8zsdr0000gn/T/ipykernel_61593/4285434580.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned.drop([1407, 2195, 2640], axis=0, inplace=True)
/var/folders/_c/dcjy1m9109bd_sq8l8j8zsdr0000gn/T/ipykernel_61593/4285434580.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/us

In [524]:
df_cleaned[df_cleaned.address.str.contains("포지리  1249", na=False)]

,cleaned_type,capacity_MW,address,year,company,owner
1843,solar,30.0,"충남 태안군 이원면 포지리 1249, 관리 1570",2020,이원신재생에너지복지마을,김철웅
2653,solar,30.0,"충남 태안군 이원면 포지리 1249, 관리 1570",2022,이원신재생에너지복지마을,신경철
